### Segmenting and Clustering - Toronto

##### Importing libraries

In [169]:
from bs4 import BeautifulSoup
import requests
import collections as co
import pandas as pd
import numpy as np
import math

In [170]:
!wget -q -O 'page.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

##### BeautifulSoup definition

In [171]:
with open('page.html','r') as fin:
    soup = BeautifulSoup(fin.read(),'lxml')
    
table = soup.find('table', attrs={'class': 'wikitable sortable'})
table_body = table.find('tbody')
table_head = table.find('tr')

header = []    
for th in table_head.findAll('th'):
    key = th.text.strip()
    header.append(key)

In [172]:
rows = len(table_body.findAll('tr'))
our_list = []

##### Reading values into list

In [173]:
i=0
for table_row in table_body.findAll('td'):
    i=i+1
    if i==1: 
        postcode = table_row.text.strip()
    elif i==2: 
        borough = table_row.text.strip()
    elif i==3: 
                neighborhood = table_row.text.strip() 
                one_row = {'Postcode': postcode, 'Borough': borough, 'Neighbourhood': neighborhood}
                our_list.append(one_row)
                i=0                

In [174]:
df = pd.DataFrame.from_records(our_list, columns=header)
df = df.rename(columns = {'Postcode':'PostCode'})
df.replace('Not assigned', np.NaN, inplace=True)
df.dropna(subset=['Borough'],inplace=True)

In [203]:
grouped = df.groupby(['PostCode','Borough'])['Neighbourhood'].apply(list).reset_index() 

,PostCode,Borough,Neighbourhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]


In [ ]:
if(grouped['Neighbourhood'].isnan):
    grouped['Neighbourhood'] = grouped['Borough']

In [234]:
grouped.shape

(103, 3)

### Geocoding section

In [236]:
!wget -q -O 'geo.csv' http://cocl.us/Geospatial_data

In [ ]:
import geocoder # import geocoder
# initialize your variable to None
lat_lng_coords = None
# loop until you get the coordinates

while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]